# Procesamiento de las zonas del aeropuerto
<hr>

<code> **Proyecto de Datos II** </code>

En este notebook procedimos a extraer otros datos relacionados con la geometría y disposición del aeropuerto. Así pues, representamos gráficamente los datos proporcionados en los archivos GeoJSON. Dado que las pistas venían dadas como combinación de varios polígonos, decidimos combinarlos y reducir a un solo rectángulo cada una de cara a simplificar en gran medida los cálculos posteriores.

Debido a la proximidad de estos puntos a pistas concretas podemos concluir que no hay puntos de espera que compartan varias pistas. Además, observamos gracias a FlightRadar24 que los aviones no cruzan por esos puntos al aterrizar, sino que toman rutas alternativas, luego los aterrizajes no añadirán ruido a los resultados en este sentido. Otra observación realizada es el hecho de que hay aviones que no se detienen en los puntos de espera si la pista está vacía generalmente, estas aeronaves no se considerarán para el entrenamiento del modelo.
	
Los puntos de espera se han ampliado a áreas circulares de 40 metros de diámetro (la figura anterior refleja estas medidas) para poder detectar correctamente aviones situados sobre ellos. No hay intersección entre puntos de espera ni con las pistas.


In [ ]:
# Importamos las librerías necesarias
import folium
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, mapping, Point
from shapely.ops import unary_union

# ============================
# 1. Procesamos el GeoJSON de pistas
# ============================

# Cargamos el GeoJSON de pistas
with open("./puntosespera/runways.geojson", 'r', encoding='utf-8') as f:
    data = json.load(f)

# Creamos diccionarios para agrupar geometrías por pista y guardar sus propiedades
runway_geometries = {}
properties_store = {}

# Recorremos cada elemento del GeoJSON
for feature in data["features"]:
    runway_name = feature["properties"].get("RWY")
    geometry = shape(feature["geometry"])

    if runway_name:
        if runway_name in runway_geometries:
            runway_geometries[runway_name].append(geometry)
        else:
            runway_geometries[runway_name] = [geometry]
            properties_store[runway_name] = feature["properties"]

# Combinamos geometrías por pista y calculamos rectángulos mínimos
combined_features = []
runway_centroids = {}

for runway_name, geometries in runway_geometries.items():
    combined_geometry = unary_union(geometries)
    min_rectangle = combined_geometry.minimum_rotated_rectangle
    runway_centroids[runway_name] = min_rectangle.centroid

    combined_features.append({
        "type": "Feature",
        "geometry": mapping(min_rectangle),
        "properties": properties_store[runway_name]
    })

# Creamos un nuevo GeoJSON con pistas representadas como rectángulos
filtered_data = {"type": "FeatureCollection", "features": combined_features}

# Guardamos el GeoJSON procesado
with open("./puntosespera/runways_rectangles.geojson", 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

# ============================
# 2. Procesamos los puntos de espera
# ============================

# Cargamos el GeoJSON de puntos de espera
with open("./puntosespera/holding_points.geojson", 'r', encoding='utf-8') as f:
    holding_data = json.load(f)

# Asignamos a cada punto de espera la pista más cercana
for feature in holding_data["features"]:
    point = Point(feature["geometry"]["coordinates"])
    closest_runway = min(runway_centroids, key=lambda rwy: point.distance(runway_centroids[rwy]))
    feature["properties"]["CLOSEST_RUNWAY"] = closest_runway

# Guardamos el GeoJSON actualizado con la pista más cercana
with open("./puntosespera/holding_points_updated.geojson", 'w', encoding='utf-8') as f:
    json.dump(holding_data, f, ensure_ascii=False, indent=4)

# ============================
# 3. Convertimos los datos a DataFrames
# ============================

# Creamos un DataFrame para las pistas
runway_data = []
for feature in filtered_data["features"]:
    properties = feature["properties"]
    geometry = shape(feature["geometry"])
    runway_data.append({
        "Runway": properties.get("RWY", "N/A"),
        "Airport": properties.get("NAME_TXT_ADHP", "N/A"),
        "Geometry": geometry
    })

df_runways = pd.DataFrame(runway_data)

# Creamos un DataFrame para los puntos de espera
holding_data_list = []
for feature in holding_data["features"]:
    properties = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    holding_data_list.append({
        "Designator": properties.get("DESIGNATOR", "N/A"),
        "Closest Runway": properties.get("CLOSEST_RUNWAY", "N/A"),
        "Remarks": properties.get("REMARKS_TXT", "N/A"),
        "Latitude": coords[1],
        "Longitude": coords[0]
    })

df_holding_points = pd.DataFrame(holding_data_list)

# ============================
# 4. Exportamos a GeoJSON
# ============================

# Guardamos las pistas procesadas en formato GeoJSON
gdf_runways = gpd.GeoDataFrame(df_runways, geometry="Geometry")
gdf_runways.to_file("runways_processed.geojson", driver="GeoJSON")

# Creamos formato de pista como "XX/YY"
df_holding_points["Runway"] = df_holding_points["Closest Runway"].map(lambda x: f"{x[-3:]}/{x[:3]}")

# Reorganizamos columnas
df_holding_points = df_holding_points[["Designator", "Runway", "Latitude", "Longitude"]]

# Creamos un GeoDataFrame para los puntos de espera
gdf_holding_points = gpd.GeoDataFrame(
    df_holding_points,
    geometry=gpd.points_from_xy(df_holding_points.Longitude, df_holding_points.Latitude)
)

# Eliminamos columnas innecesarias
gdf_holding_points = gdf_holding_points.drop(columns=["Longitude", "Latitude"])

# Guardamos el resultado en GeoJSON
gdf_holding_points.to_file("holding_points_processed.geojson", driver="GeoJSON")
